<a href="https://colab.research.google.com/github/ak2742/text-tagging-with-llama3/blob/main/llama3_tagging_gradio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Install package and load the extension
!pip install colab-xterm
%load_ext colabxterm
%xterm

# #- Install/run Ollama in the terminal

# !curl -fsSL https://ollama.com/install.sh | sh
# !ollama serve & ollama pull llama3

In [ ]:
#@title Install libraries

#LangChain related libraries
!pip install -q -U langchain langchain_experimental

!pip install -q -U gradio

In [ ]:
#@title tagging config

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field

# tagging prompt template
tagging_prompt = ChatPromptTemplate.from_template(
"""
Extract the desired information from the  passage given in triple Backticks.

Only extract the properties mentioned in the 'Classification' function.

Please follow the response format as given.

Passage:
'''{input}'''
"""
)

# Output Schema for structured response
class Classification(BaseModel):
    sentiment: str = Field(description="The sentiment of the text")
    aggressiveness: int = Field(
        description="How aggressive the text is on a scale from 1 to 10"
    )
    language: str = Field(description="The language the text is written in")
    confidence_score: float = Field(
        default=None, description="Confidence in sentiment prediction (0-1)"
    )
    topic: str = Field(default=None, description="The main topic of the text")
    keywords: list[str] = Field(default=None, description="List of keywords found")


In [ ]:
#@title Create Chain with llama3

from langchain_experimental.llms.ollama_functions import OllamaFunctions

# Chain
llm = OllamaFunctions(model="llama3", format="json", temperature=0)
structured_llm = llm.with_structured_output(Classification)
chain = tagging_prompt | structured_llm

In [ ]:
#@title Add Gradio UI

import gradio as gr

def gradio_fn(msg):
    response = chain.invoke({"input": msg}).dict()
    return response

gr.Interface(
    fn=gradio_fn,
    inputs=gr.Textbox(lines=4, label="Input Text"),
    outputs=gr.Textbox(lines=4, label="Output Data"),
    theme="soft"
    ).launch(share=True, debug=True)